In [ ]:
cd /content/drive/My Drive

In [30]:
import pandas as pd
import datetime as dt
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import KFold, cross_val_score
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras import backend as K
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

In [31]:
data = pd.read_excel( 'Modified Dataset.xls' )

In [32]:
data.head(10)

,id,Miangin_Ketab,Toole_Ketab,Miangin_Gheimat,Majmoo_Gheimat,Review,Emtiaze_review,Completion,Daghayeghe-Shenide shode,Darkhaste-Poshtibani,Zamane_akharin_bazdid -Zamane_kharid,Target
0,9011,648.0,648,5.33,5.33,0,NaN,0.0,0.0,0,0,1
1,16370,2160.0,2160,5.33,5.33,0,NaN,0.0,0.0,0,0,1
2,19996,594.0,1188,29.33,58.67,0,10.0,0.0,0.0,0,0,1
3,27030,1188.0,4752,6.00,23.99,0,10.0,0.0,0.0,0,176,1
4,29095,1404.0,2808,27.99,55.99,0,NaN,0.0,0.0,0,0,1
5,3699,2160.0,2160,5.33,5.33,0,NaN,0.0,0.0,0,1,1
6,4722,1332.0,3996,5.33,15.99,0,NaN,0.0,0.0,0,86,1
7,12413,756.0,1512,8.73,17.47,0,NaN,0.0,0.0,0,0,1
8,17321,1404.0,2808,11.35,22.70,0,10.0,0.0,0.0,0,209,1
9,18755,2160.0,2160,10.98,10.98,0,NaN,0.0,0.0,0,351,1


In [33]:
#Determine Missing values
print("missing value")
print(data.isna().sum())

missing value
id                                         0
Miangin_Ketab                              0
Toole_Ketab                                0
Miangin_Gheimat                            0
Majmoo_Gheimat                             0
Review                                     0
Emtiaze_review                          3499
Completion                                 0
Daghayeghe-Shenide shode                   0
Darkhaste-Poshtibani                       0
Zamane_akharin_bazdid -Zamane_kharid       0
Target                                     0
dtype: int64


In [5]:
# Fill missing Values with zero
data = data.fillna(0)

In [6]:
# Drop High Correlated features
data.drop('id', inplace=True, axis=1)
data.drop('Emtiaze_review', inplace=True, axis=1)
data.drop('Miangin_Gheimat', inplace=True, axis=1)
data.drop('Miangin_Ketab', inplace=True, axis=1)
data.drop('Daghayeghe-Shenide shode', inplace=True, axis=1)

In [7]:
#showing that the dataset is no longer imbalanced.
data['Target'].value_counts()

1    2237
0    2237
Name: Target, dtype: int64

In [8]:
#shuffling the dataset
data = data.sample(frac = 1,ignore_index = True) #shuffling the dataset

In [9]:
data.head(10)

,Toole_Ketab,Majmoo_Gheimat,Review,Completion,Darkhaste-Poshtibani,Zamane_akharin_bazdid -Zamane_kharid,Target
0,2160,5.33,0,0.03,0,8,0
1,1188,7.47,0,0.00,0,31,1
2,2160,5.33,0,0.00,0,0,1
3,1620,7.99,1,0.00,0,5,1
4,2160,7.99,0,0.00,0,54,0
5,2160,5.33,0,0.00,0,0,1
6,2160,10.13,0,0.00,0,0,1
7,1620,8.00,0,0.00,0,98,1
8,3348,16.00,0,0.00,0,0,1
9,1620,5.74,0,0.14,0,202,0


In [10]:
x = data. iloc[:, 0:6]

In [11]:
y = data['Target']

In [12]:
x

,Toole_Ketab,Majmoo_Gheimat,Review,Completion,Darkhaste-Poshtibani,Zamane_akharin_bazdid -Zamane_kharid
0,2160,5.33,0,0.03,0,8
1,1188,7.47,0,0.00,0,31
2,2160,5.33,0,0.00,0,0
3,1620,7.99,1,0.00,0,5
4,2160,7.99,0,0.00,0,54
...,...,...,...,...,...,...
4469,1620,8.00,0,0.00,0,35
4470,3780,10.67,0,0.00,0,0
4471,5076,23.47,0,0.00,0,0
4472,1620,5.33,0,0.00,0,162


In [13]:
y

0       0
1       1
2       1
3       1
4       0
       ..
4469    1
4470    1
4471    1
4472    1
4473    0
Name: Target, Length: 4474, dtype: int64

In [15]:
#Using MinMax Scaler
scaler = MinMaxScaler() 
scaled_x = scaler.fit_transform(x) 

In [16]:
print(scaled_x )

[[0.28571429 0.01366044 0.         0.03       0.         0.02139037]
 [0.14285714 0.03354707 0.         0.         0.         0.0828877 ]
 [0.28571429 0.01366044 0.         0.         0.         0.        ]
 ...
 [0.71428571 0.18223213 0.         0.         0.         0.        ]
 [0.20634921 0.01366044 0.         0.         0.         0.43315508]
 [0.28571429 0.01366044 0.         0.         0.         0.        ]]


**Cross Validation with Naive Bayes , Decision Tree and Deep Learning**

**Naive Bayes**

In [17]:
#10 Folds
k_folds = KFold(n_splits = 10)

In [18]:
NBmodel = MultinomialNB()
DTmodel = DecisionTreeClassifier(criterion="entropy")

In [19]:
#Naive Bayes
scores = cross_val_score(NBmodel, scaled_x , y, scoring="accuracy", cv = k_folds)
print("Cross Validation Accuracy using Naive Bayes Classifier : ", scores)
print("Average CV Accuracy: ", scores.mean())

Cross Validation Accuracy using Naive Bayes Classifier :  [0.72098214 0.734375   0.71205357 0.74107143 0.7360179  0.72706935
 0.7606264  0.78076063 0.70917226 0.7360179 ]
Average CV Accuracy:  0.7358146572387344


In [21]:
scores = cross_val_score(NBmodel, scaled_x , y, scoring="recall", cv = k_folds)
print("Cross Validation Recall using Naive Bayes Classifier: ", scores)
print("Average CV Recall: ", scores.mean())

Cross Validation Recall using Naive Bayes Classifier:  [0.98214286 0.99561404 0.98237885 0.99115044 1.         0.98636364
 0.9795082  0.9874477  1.         1.        ]
Average CV Recall:  0.9904605721163721


In [22]:
scores = cross_val_score(NBmodel, scaled_x , y, scoring="precision", cv = k_folds)
print("Cross Validation Precision using Naive Bayes Classifier:  ", scores)
print("Average CV Precision: ", scores.mean())

Cross Validation Precision using Naive Bayes Classifier:   [0.64516129 0.65797101 0.6408046  0.66272189 0.63239875 0.64583333
 0.70087977 0.71299094 0.62099125 0.64350453]
Average CV Precision:  0.6563257370553177


In [35]:
scores = cross_val_score(NBmodel, scaled_x , y, scoring="f1", cv = k_folds)
print("Cross Validation f1 using Naive Bayes Classifier: ", scores)
print("Average CV f1: ", scores.mean())

Cross Validation f1 using Naive Bayes Classifier:  [0.77876106 0.79232112 0.77565217 0.79432624 0.77480916 0.78057554
 0.81709402 0.82807018 0.76618705 0.78308824]
Average CV f1:  0.7890884771983278


**Decision Tree**

In [23]:
#Decision Tree
scores = cross_val_score(DTmodel, scaled_x, y, scoring="accuracy", cv = k_folds)
print("Cross Validation Accuracy: ", scores)
print("Average CV Accuracy: ", scores.mean())

Cross Validation Accuracy:  [0.78348214 0.82589286 0.828125   0.85044643 0.81655481 0.84116331
 0.83221477 0.81879195 0.84116331 0.86129754]
Average CV Accuracy:  0.8299132110898052


In [24]:
scores = cross_val_score(DTmodel, scaled_x , y, scoring="recall", cv = k_folds)
print("Cross Validation Recall using Decision Tree: ", scores)
print("Average CV Recall: ", scores.mean())

Cross Validation Recall using Decision Tree:  [0.70982143 0.75877193 0.78414097 0.83628319 0.76847291 0.78181818
 0.79508197 0.76987448 0.82159624 0.82159624]
Average CV Recall:  0.7847457534085288


In [25]:
scores = cross_val_score(DTmodel, scaled_x , y, scoring="precision", cv = k_folds)
print("Cross Validation Precision using Decision Tree: ", scores)
print("Average CV Precision: ", scores.mean())

Cross Validation Precision using Decision Tree:  [0.8342246  0.87692308 0.85645933 0.86238532 0.82010582 0.87817259
 0.88584475 0.87323944 0.84134615 0.87309645]
Average CV Precision:  0.8601797522061151


In [26]:
scores = cross_val_score(DTmodel, scaled_x, y, scoring="f1", cv = k_folds)
print("Cross Validation F1 using Decision Tree: ", scores)
print("Average CV F1: ", scores.mean())

Cross Validation F1 using Decision Tree:  [0.76328502 0.8056872  0.82298851 0.8494382  0.78772379 0.82692308
 0.84233261 0.82300885 0.83333333 0.84466019]
Average CV F1:  0.8199380788486235


**Deep Learning**

In [27]:
#Deep Learning Manual Claculation 
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [28]:
#Deep Learning With Cross Validation
Dlcvscores1 = []
Dlcvscores2 = []
Dlcvscores3 = []
Dlcvscores4 = []
for train, test in k_folds.split(scaled_x, y):
 # create model
 model = Sequential()
 model.add(Dense(10, input_dim=6, activation='relu'))
 model.add(Dense(10, activation='relu'))
 model.add(Dense(1, activation='sigmoid'))
 # Compile model
 model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc',f1_m,precision_m, recall_m])
 # Fit the model
 model.fit(scaled_x[train], y[train], epochs=10, batch_size=10, verbose=0)
 # evaluate the model
 loss, accuracy, f1_score, precision, recall = model.evaluate(scaled_x[test], y[test], verbose=0)
 print("%s: %.2f%%" % ('accuracy', accuracy*100))
 print("%s: %.2f%%" % ('f1_score', f1_score*100))
 print("%s: %.2f%%" % ('precision', precision*100))
 print("%s: %.2f%%" % ('recall', recall*100))
 print('Waiting for other folds to be taken as test set')
Dlcvscores1.append(accuracy * 100)
Dlcvscores2.append(f1_score * 100)
Dlcvscores3.append(precision * 100)
Dlcvscores4.append(recall * 100)

print("Average CV Accuracy (Cross Validation): ", (np.mean(Dlcvscores1)))
print("Average CV F1 (Cross Validation): ", (np.mean(Dlcvscores2)))
print("Average CV Precision (Cross Validation) : ", (np.mean(Dlcvscores3)))
print("Average CV Recall (Cross Validation) : ", (np.mean(Dlcvscores4)))

accuracy: 78.12%
f1_score: 78.08%
precision: 79.90%
recall: 77.01%
Waiting for other folds to be taken as test set
accuracy: 78.12%
f1_score: 79.49%
precision: 75.12%
recall: 84.99%
Waiting for other folds to be taken as test set
accuracy: 80.36%
f1_score: 79.43%
precision: 82.54%
recall: 77.34%
Waiting for other folds to be taken as test set
accuracy: 83.48%
f1_score: 82.45%
precision: 89.95%
recall: 77.12%
Waiting for other folds to be taken as test set
accuracy: 77.63%
f1_score: 74.03%
precision: 75.13%
recall: 74.87%
Waiting for other folds to be taken as test set
accuracy: 80.76%
f1_score: 77.28%
precision: 88.28%
recall: 69.50%
Waiting for other folds to be taken as test set
accuracy: 79.87%
f1_score: 79.71%
precision: 86.85%
recall: 74.53%
Waiting for other folds to be taken as test set
accuracy: 79.42%
f1_score: 77.70%
precision: 86.75%
recall: 70.99%
Waiting for other folds to be taken as test set
accuracy: 80.98%
f1_score: 79.08%
precision: 83.99%
recall: 76.96%
Waiting for o

**70/30 approach**

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x_scaled,y, stratify=y, test_size=0.3, random_state=None)

**Naive Bayes Classifier**

In [ ]:
modelNB = MultinomialNB()
modelNB.fit(x_train, y_train)

MultinomialNB()

In [ ]:
y_predict = modelNB.predict(x_test)

In [ ]:
print('Accuracy: %.3f' % accuracy_score(y_test, y_predict))
print('Recall: %.3f' % recall_score(y_test, y_predict))
print('Precision: %.3f' % precision_score(y_test, y_predict))
print('F1: %.3f' % f1_score(y_test, y_predict))

Accuracy: 0.711
Recall: 0.958
Precision: 0.641
F1: 0.768


**Decision Tree**

In [ ]:
modelDT = DecisionTreeClassifier(criterion="entropy")
modelDT.fit(x_train, y_train)

DecisionTreeClassifier(criterion='entropy')

In [ ]:
y_predict2 = modelDT.predict(x_test)

In [ ]:
print('Accuracy: %.3f' % accuracy_score(y_test, y_predict2))
print('Recall: %.3f' % recall_score(y_test, y_predict2))
print('Precision: %.3f' % precision_score(y_test, y_predict2))
print('F1: %.3f' % f1_score(y_test, y_predict2))

Accuracy: 0.815
Recall: 0.781
Precision: 0.839
F1: 0.809


**Deep Learning**

In [ ]:
modelDL = Sequential()
modelDL.add(Dense(8, input_dim=6, activation='relu'))
modelDL.add(Dense(6, activation='relu'))
modelDL.add(Dense(4, activation='relu'))
modelDL.add(Dense(1, activation='sigmoid'))
# Compile model
modelDL.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc',f1_m,precision_m, recall_m])
# Fit the model
modelDL.fit(x_train, y_train, epochs=20, batch_size=10, verbose=1)

Epoch 1/20
314/314 [==============================] - 2s 2ms/step - loss: 0.6580 - acc: 0.5733 - f1_m: 0.2619 - precision_m: 0.4518 - recall_m: 0.2060
Epoch 2/20
314/314 [==============================] - 1s 2ms/step - loss: 0.5682 - acc: 0.7288 - f1_m: 0.6461 - precision_m: 0.8037 - recall_m: 0.5900
Epoch 3/20
314/314 [==============================] - 1s 2ms/step - loss: 0.4614 - acc: 0.7889 - f1_m: 0.7613 - precision_m: 0.8149 - recall_m: 0.7614
Epoch 4/20
314/314 [==============================] - 1s 3ms/step - loss: 0.4046 - acc: 0.7981 - f1_m: 0.7753 - precision_m: 0.8381 - recall_m: 0.7575
Epoch 5/20
314/314 [==============================] - 1s 4ms/step - loss: 0.3804 - acc: 0.7997 - f1_m: 0.7715 - precision_m: 0.8140 - recall_m: 0.7680
Epoch 6/20
314/314 [==============================] - 1s 3ms/step - loss: 0.3677 - acc: 0.8023 - f1_m: 0.7743 - precision_m: 0.8231 - recall_m: 0.7710
Epoch 7/20
314/314 [==============================] - 1s 4ms/step - loss: 0.3597 - acc: 0.8055

In [ ]:

loss, accuracy, f1_score, precision, recall = modelDL.evaluate(x_test, y_test, verbose=0)

In [ ]:
print('Accuracy: ', accuracy)
print('Recall: ', recall)
print('Precision: ', precision)
print('F1: ', f1_score)

Accuracy:  0.8078927993774414
Recall:  0.7769321203231812
Precision:  0.8286899328231812
F1:  0.7974483966827393
